# Training and Evaluation of GNNs and LLMs
In this notebook, we train the models on the [MovieLens Dataset](https://movielens.org/) after the Pytorch Geometrics Tutorial on [Link Prediction](https://colab.research.google.com/drive/1xpzn1Nvai1ygd_P5Yambc_oe4VBPK_ZT?usp=sharing#scrollTo=vit8xKCiXAue).

First we import all of our dependencies.

The **GraphRepresentationGenerator** manages and trains a GNN model. Its most important interfaces include
**the constructor**, which defines the GNN architecture and loads the pre-trained GNN model if it is already on the hard disk,
**the training method**, which initializes the training on the GNN model and
**the get_embedding methods**, which represent the inference interface to the GNN model and return the corresponding embeddings in the dimension defined in the constructor for given user movie node pairs.

**The MovieLensLoader** loads and manages the data sets. The most important tasks include **saving and (re)loading and transforming** the data sets.

**PromptEncoderOnlyClassifier** and **VanillaEncoderOnlyClassifier** each manage a **prompt (model) LLM** and a **vanilla (model) LLM**. An EncoderOnlyClassifier (ClassifierBase) provides interfaces for training and testing an LLM model.
PromptEncoder and VanillaEncoder differ from their DataCollectors. DataCollectors change the behavior of the models during training and testing and allow data points to be created at runtime. With the help of these collators, we **create non-existent edges on the fly**.

In [1]:
from graph_representation_generator.graph_representation_generator import (
    GraphRepresentationGenerator,
)
from dataset_manager.movie_lens_manager import MovieLensManager
from dataset_manager.kg_manager import ROOT
from llm_manager.graph_prompter_hf.classifier import GraphPrompterHF

In [2]:
MODEL_NAME = "google/bert_uncased_L-2_H-128_A-2"
EPOCHS = 20
BATCH_SIZE_LLM = 256

We define in advance which **Knowledge Graph Embedding Dimension (KGE_DIMENSION)** the GNN encoder has. We want to determine from which output dimension the GNN encoder can produce embeddings that lead to a significant increase in performance *without exceeding the context length of the LLMs*. In the original tutorial, the KGE_DIMENSION was $64$.

In [3]:
kg_manager = MovieLensManager()

llm_df = kg_manager.llm_df.merge(kg_manager.target_df[["id", "prompt_feature_title", "prompt_feature_genres"]].rename(columns={"id": "target_id"}), on = "target_id")
llm_df

First we load the MovieLensLoader, which downloads the Movie Lens dataset (https://files.grouplens.org/datasets/movielens/ml-32m.zip) and prepares it to be used on GNN and LLM. We also pass the embedding dimensions that we will assume we are training with. First time takes approx. 30 sec.

In [4]:
kg_manager.data

HeteroData(
  source={ node_id=[610] },
  target={
    node_id=[9742],
    x=[9742, 20],
  },
  (source, edge, target)={ edge_index=[2, 100836] },
  (target, rev_edge, source)={ edge_index=[2, 100836] }
)

Next, we initialize the GNN trainers (possible on Cuda), one for each KGE_DIMENSION.
A GNN trainer manages a model and each model consists of an **encoder and classifier** part.

**The encoder** is a parameterized *Grap Convolutional Network (GCN)* with a *2-layer GNN computation graph* and a single *ReLU* activation function in between.

**The classifier** applies the dot-product between source and destination kges to derive edge-level predictions.

In [5]:
graph_representation_generator_graph_prompter_hf = GraphRepresentationGenerator(
    kg_manager.data.to("cuda"),
    kg_manager.gnn_train_data.to("cuda"),
    kg_manager.gnn_val_data.to("cuda"),
    kg_manager.gnn_test_data.to("cuda"),
    hidden_channels=128,
    kge_dimension=128,
    force_recompute=False,
    device="cuda",
)

loading pretrained model
Device: 'cuda'


Next we produce the KGEs for every edge in the dataset. These embeddings can then be used for the LLM on the link-prediction task.

Next we initialize the vanilla encoder only classifier. This classifier does only use the NLP part of the prompt (no KGE) for predicting if the given link exists.

In [6]:
graph_prompter_hf_bert_classifier = GraphPrompterHF(
    kge_manager=kg_manager,
    get_embeddings_cb=graph_representation_generator_graph_prompter_hf.get_embeddings,
    model_name=MODEL_NAME,
    vanilla_model_path=f"{ROOT}/llm/graph_prompter_hf_frozen/training/best",
    gnn_parameters=list(
        graph_representation_generator_graph_prompter_hf.model.parameters()
    ),
)

device cuda
5
5
5


In [7]:
dataset_embedding = kg_manager.generate_graph_prompter_hf_embedding_dataset(
    graph_prompter_hf_bert_classifier.tokenizer.sep_token,
    graph_prompter_hf_bert_classifier.tokenizer.pad_token,
    graph_prompter_hf_bert_classifier.tokenize_function,
)

In [8]:
graph_prompter_hf_bert_classifier.model.device

device(type='cuda', index=0)

In [9]:
graph_prompter_hf_bert_classifier.train_model_on_data(
    dataset_embedding,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE_LLM,
)
graph_representation_generator_graph_prompter_hf.save_model()

Adding GNN parameters to optimizer


  0%|          | 0/4420 [00:00<?, ?it/s]

c:\Users\MARS\.conda\envs\hauptprojekt\Lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


{'loss': 0.355, 'grad_norm': 1.0741111040115356, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.05}
{'loss': 0.3517, 'grad_norm': 1.876395344734192, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.09}
{'loss': 0.3609, 'grad_norm': 0.9085135459899902, 'learning_rate': 3e-06, 'epoch': 0.14}
{'loss': 0.3533, 'grad_norm': 2.4121317863464355, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.18}
{'loss': 0.3715, 'grad_norm': 1.5719934701919556, 'learning_rate': 5e-06, 'epoch': 0.23}
{'loss': 0.3641, 'grad_norm': 3.3062729835510254, 'learning_rate': 6e-06, 'epoch': 0.27}
{'loss': 0.3394, 'grad_norm': 1.954710602760315, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.32}
{'loss': 0.3445, 'grad_norm': 0.876276969909668, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.36}
{'loss': 0.348, 'grad_norm': 3.0029659271240234, 'learning_rate': 9e-06, 'epoch': 0.41}
{'loss': 0.3726, 'grad_norm': 1.1106078624725342, 'learning_rate': 1e-05, 'epoch': 0.45}
{'loss': 0.3467, 'grad_norm': 2.5

  0%|          | 0/79 [00:00<?, ?it/s]

...

{'eval_loss': 0.39922094345092773, 'eval_accuracy': 0.8283249033025886, 'eval_runtime': 24.6248, 'eval_samples_per_second': 818.931, 'eval_steps_per_second': 3.208, 'epoch': 18.0}
{'loss': 0.2776, 'grad_norm': 0.7784190773963928, 'learning_rate': 5.612244897959184e-06, 'epoch': 18.01}
{'loss': 0.2789, 'grad_norm': 0.6871457695960999, 'learning_rate': 5.48469387755102e-06, 'epoch': 18.05}
{'loss': 0.2708, 'grad_norm': 1.122105360031128, 'learning_rate': 5.357142857142857e-06, 'epoch': 18.1}
{'loss': 0.267, 'grad_norm': 0.6641886830329895, 'learning_rate': 5.229591836734695e-06, 'epoch': 18.14}
{'loss': 0.2917, 'grad_norm': 0.7204041481018066, 'learning_rate': 5.102040816326531e-06, 'epoch': 18.19}
{'loss': 0.2441, 'grad_norm': 0.888738214969635, 'learning_rate': 4.9744897959183674e-06, 'epoch': 18.24}
{'loss': 0.2654, 'grad_norm': 0.6214226484298706, 'learning_rate': 4.846938775510204e-06, 'epoch': 18.28}
{'loss': 0.2696, 'grad_norm': 0.7031832933425903, 'learning_rate': 4.7193877551020

  0%|          | 0/79 [00:00<?, ?it/s]

{'eval_loss': 0.38471725583076477, 'eval_accuracy': 0.8325399186749975, 'eval_runtime': 25.1758, 'eval_samples_per_second': 801.008, 'eval_steps_per_second': 3.138, 'epoch': 19.0}
{'loss': 0.2811, 'grad_norm': 0.7393852472305298, 'learning_rate': 2.806122448979592e-06, 'epoch': 19.0}
{'loss': 0.2738, 'grad_norm': 1.002983570098877, 'learning_rate': 2.6785714285714285e-06, 'epoch': 19.05}
{'loss': 0.2818, 'grad_norm': 0.7903978824615479, 'learning_rate': 2.5510204081632653e-06, 'epoch': 19.1}
{'loss': 0.272, 'grad_norm': 0.5809009671211243, 'learning_rate': 2.423469387755102e-06, 'epoch': 19.14}
{'loss': 0.2502, 'grad_norm': 0.721860945224762, 'learning_rate': 2.295918367346939e-06, 'epoch': 19.19}
{'loss': 0.2747, 'grad_norm': 0.9174560308456421, 'learning_rate': 2.1683673469387757e-06, 'epoch': 19.23}
{'loss': 0.2612, 'grad_norm': 0.7184321284294128, 'learning_rate': 2.040816326530612e-06, 'epoch': 19.28}
{'loss': 0.2779, 'grad_norm': 0.7707904577255249, 'learning_rate': 1.91326530612

  0%|          | 0/79 [00:00<?, ?it/s]

{'eval_loss': 0.39049065113067627, 'eval_accuracy': 0.8331349796687494, 'eval_runtime': 24.9873, 'eval_samples_per_second': 807.051, 'eval_steps_per_second': 3.162, 'epoch': 20.0}
{'train_runtime': 2293.9303, 'train_samples_per_second': 492.334, 'train_steps_per_second': 1.927, 'train_loss': 0.2993068355240973, 'epoch': 20.0}


In [10]:
import torch

tens_1 = graph_representation_generator_graph_prompter_hf.model.to(
    device="cpu"
).state_dict()

tens_2 = torch.load("./data/gnn/backup/model_128.pth")
for key in tens_1.keys():
    if isinstance(tens_1[key], torch.Tensor):
        if (tens_1[key] == tens_2[key]).all():
            print(key, "is same")
        else:
            print(key, "is not same")

target_lin.weight is not same
target_lin.bias is not same
source_emb.weight is not same
target_emb.weight is not same
gnn.conv1.source__edge__target.lin_l.weight is not same
gnn.conv1.source__edge__target.lin_l.bias is not same
gnn.conv1.source__edge__target.lin_r.weight is not same
gnn.conv1.target__rev_edge__source.lin_l.weight is not same
gnn.conv1.target__rev_edge__source.lin_l.bias is not same
gnn.conv1.target__rev_edge__source.lin_r.weight is not same
gnn.conv2.source__edge__target.lin_l.weight is not same
gnn.conv2.source__edge__target.lin_l.bias is not same
gnn.conv2.source__edge__target.lin_r.weight is not same
gnn.conv2.target__rev_edge__source.lin_l.weight is not same
gnn.conv2.target__rev_edge__source.lin_l.bias is not same
gnn.conv2.target__rev_edge__source.lin_r.weight is not same
